In [2]:
import torch

In [3]:
from torch import nn

In [4]:
from torch.utils.data import DataLoader

In [5]:
from torchvision import datasets

In [6]:
from torchvision.transforms import ToTensor

In [7]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

In [11]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, label

# Train 데이터셋 경로 설정
train_dataset_root = r'C:\\Users\\내문서\\Desktop\\kaggle\\training_set'

# Test 데이터셋 경로 설정
test_dataset_root = r'C:\\Users\\내문서\\Desktop\\kaggle\\test_set'

# Train 이미지 파일과 레이블을 읽어옴
train_images = []
train_labels = []

for label, class_name in enumerate(os.listdir(train_dataset_root)):
    class_path = os.path.join(train_dataset_root, class_name)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            train_images.append(img_path)
            train_labels.append(label)

# Test 이미지 파일과 레이블을 읽어옴
test_images = []
test_labels = []

for label, class_name in enumerate(os.listdir(test_dataset_root)):
    class_path = os.path.join(test_dataset_root, class_name)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            test_images.append(img_path)
            test_labels.append(label)

## 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # 원하는 크기로 조정
    transforms.ToTensor()
])

# Train 데이터셋 생성
train_dataset = CustomDataset(train_images, train_labels, transform=transform)

# Test 데이터셋 생성
test_dataset = CustomDataset(test_images, test_labels, transform=transform)

# # 데이터로더 생성
# batch_size = 64
# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

batch_size = 64
print(len(train_dataset))
print(len(test_dataset))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for X, y in test_dataloader:
    print(f"Shape of X [N,C,H,W]:{X.shape}" )
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

8005
2023
Shape of X [N,C,H,W]:torch.Size([64, 3, 64, 64])
Shape of y: torch.Size([64]) torch.int64


In [77]:
# FashionMNIST에서 데이터를 가져온 후 테스트

# training_data = datasets.FashionMNIST(
#     root = "data", train = True, download =True,transform = ToTensor(),
# )
# 
# test_data = datasets.FashionMNIST(
#     root = "data", train = False,download=True,transform = ToTensor(),
# )

In [12]:
# cuda를 사용한 gpu

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [13]:
# 신경망 구축

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        
        layers = []
        sizes = [input_size] + hidden_sizes + [output_size]
        
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if i < len(sizes) - 2:  # Add ReLU activation except for the last layer
                layers.append(nn.ReLU())
        
        self.linear_relu_stack = nn.Sequential(*layers)

    def forward(self, x):
        x = self.flatten(x)
        logit = self.linear_relu_stack(x)
        return logit

# 이미지 크기와 클래스 수에 맞게 모델을 초기화
input_size = 64 * 64 * 3  
hidden_sizes = [1440, 1440, 1000, 1000, 1000, 784, 784, 324, 64] 
output_size = 2  # 클래스 수에 따라 조절

# 모델 정의
model = NeuralNetwork(input_size, hidden_sizes, output_size).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=12288, out_features=1440, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1440, out_features=1440, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1440, out_features=1000, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1000, out_features=1000, bias=True)
    (7): ReLU()
    (8): Linear(in_features=1000, out_features=1000, bias=True)
    (9): ReLU()
    (10): Linear(in_features=1000, out_features=784, bias=True)
    (11): ReLU()
    (12): Linear(in_features=784, out_features=784, bias=True)
    (13): ReLU()
    (14): Linear(in_features=784, out_features=324, bias=True)
    (15): ReLU()
    (16): Linear(in_features=324, out_features=64, bias=True)
    (17): ReLU()
    (18): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [14]:
# loss function 정의
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        
        pred = model(X)
        loss= loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'Loss: {loss:7f}[{current:>5d}/{size:>5d}]')

In [16]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # 꼭 넣어야한다.
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"test Error : \n Accruacy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [18]:
epochs =10
for t in range(epochs):
    print(f"Epoch {t+1} \n=============================")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model,loss_fn)
    optimizer.step()
    
print("Done!")

Epoch 1 
Loss: 0.696766[    0/ 8005]
Loss: 0.699927[ 6400/ 8005]
test Error : 
 Accruacy: 50.0%, Avg loss: 0.695306

Epoch 2 
Loss: 0.692910[    0/ 8005]
Loss: 0.694432[ 6400/ 8005]
test Error : 
 Accruacy: 50.0%, Avg loss: 0.695026

Epoch 3 
Loss: 0.702267[    0/ 8005]
Loss: 0.683974[ 6400/ 8005]
test Error : 
 Accruacy: 50.0%, Avg loss: 0.694772

Epoch 4 
Loss: 0.691350[    0/ 8005]
Loss: 0.687425[ 6400/ 8005]
test Error : 
 Accruacy: 50.0%, Avg loss: 0.694525

Epoch 5 
Loss: 0.708151[    0/ 8005]
Loss: 0.689090[ 6400/ 8005]
test Error : 
 Accruacy: 50.0%, Avg loss: 0.694353

Epoch 6 
Loss: 0.692686[    0/ 8005]
Loss: 0.699495[ 6400/ 8005]
test Error : 
 Accruacy: 50.0%, Avg loss: 0.694215

Epoch 7 
Loss: 0.689356[    0/ 8005]
Loss: 0.695752[ 6400/ 8005]
test Error : 
 Accruacy: 50.0%, Avg loss: 0.694094

Epoch 8 
Loss: 0.700853[    0/ 8005]
Loss: 0.689809[ 6400/ 8005]
test Error : 
 Accruacy: 50.0%, Avg loss: 0.693990

Epoch 9 
Loss: 0.691707[    0/ 8005]
Loss: 0.698875[ 6400/ 8005]